In [1]:
from gene_level_MoM import *
import zipfile

In [2]:
###unzip file
zip_files = [
    "test_data/chr1_Contiguous1KSNP.zip",
    "test_data/chr1_Random1KSNP.zip"
]

for zf in zip_files:
    with zipfile.ZipFile(zf, "r") as z:
        z.extractall("test_data")   
    print(f"Unzipped: {zf}")


Unzipped: test_data/chr1_Contiguous1KSNP.zip
Unzipped: test_data/chr1_Random1KSNP.zip


In [3]:
##read and process data
##random snp
RandomSNP = pd.read_csv("test_data/chr1_Random1KSNP.raw", sep=r'\s+')
RandomSNP = RandomSNP.iloc[:, 6:].to_numpy()
##Contiguous snp
ContiguousSNP = pd.read_csv("test_data/chr1_Contiguous1KSNP.raw", sep=r'\s+')
ContiguousSNP = ContiguousSNP.iloc[:, 6:].to_numpy()

In [4]:
##Get weight
X1 = ContiguousSNP[:2000,:200] ###use a subset to test
Z = (X1 - X1.mean(axis=0)) / X1.std(axis=0)

# Compute weights
w = weight_vector(X1)

##### see whether the weight change the distribution of var(zizj)
print("\nWithout weights:")
print(check_weighted_products(Z, 1)[2])

print("\nWith weights:")
print(check_weighted_products(Z, w)[2])


Without weights:
{'offdiag_mean': np.float64(1.248402997944833), 'offdiag_std': np.float64(1.8115280933612155), 'n_pairs': 19900}

With weights:
{'offdiag_mean': np.float64(1.4283325953506811), 'offdiag_std': np.float64(2.385414125797485), 'n_pairs': 19900}


In [5]:
import numpy as np

# Get dimensions
n, m = Z.shape

# Build the least-squares system
p = m * (m - 1) // 2
A = np.zeros((p, m))
b = np.zeros(p)

row = 0
for i in range(m):
    for j in range(i + 1, m):
        A[row, i] = 1
        A[row, j] = 1
        v = (Z[:, i] * Z[:, j]).var()
        b[row] = -0.5 * np.log(max(v, 1e-10))
        row += 1

# Compute LS solution
u_ls, *_ = np.linalg.lstsq(A, b, rcond=None)

# Objective 1: Log-space
def log_space_objective(u):
    total = 0
    for i in range(m):
        for j in range(i + 1, m):
            v = (Z[:, i] * Z[:, j]).var()
            c_ij = -0.5 * np.log(max(v, 1e-10))
            total += (u[i] + u[j] - c_ij) ** 2
    return total

# Objective 2: Original-space
def original_space_objective(u):
    w = np.exp(u)
    Zw = Z * w
    total = 0
    for i in range(m):
        for j in range(i + 1, m):
            v = (Zw[:, i] * Zw[:, j]).var()
            total += (v - 1) ** 2
    return total

u_unweighted = np.zeros(m)  # corresponds to w = 1

print("=== Log-space objective ===")
print(f"Unweighted: {log_space_objective(u_unweighted):.4f}")
print(f"LS weights: {log_space_objective(u_ls):.4f}")

print("\n=== Original-space objective ===")
print(f"Unweighted: {original_space_objective(u_unweighted):.4f}")
print(f"LS weights: {original_space_objective(u_ls):.4f}")

=== Log-space objective ===
Unweighted: 4328.8746
LS weights: 3379.2769

=== Original-space objective ===
Unweighted: 66532.4278
LS weights: 400659.8330
